In [1]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json
import requests
from datetime import datetime


In [2]:
spark=SparkSession.builder.appName('test').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/16 22:07:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import csv
import requests


url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=15min&slice=year1month1&apikey=268GM69EAKB8RIMR'

with requests.Session() as output:
    download = output.get(url)
    csv_file = download.content.decode('utf-8')
    read = csv.reader(csv_file.splitlines(), delimiter=',')
    my_list = list(read)
    df = spark.createDataFrame(my_list[1:]).toDF(*my_list[0]) 
    df = df.select(to_date(col("time")).alias("day"), col("open"), col("high"), col("low"), col("close"), col("volume"))

In [4]:
df.toPandas().to_csv('op.csv', index=False)